In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


### Package Import

In [ ]:
import os
import cv2
import glob
import os
import numpy as np
from random import shuffle
from tqdm import tqdm
import random
import shutil 
from google.colab.patches import cv2_imshow
import math
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from preprocessing import *

In [ ]:
!unzip -o /content/drive/MyDrive/edge-connect/data_split256.zip

In [ ]:
!unzip -o /content/drive/MyDrive/edge-connect/test.zip

In [ ]:
# !cp /content/data_split256/test/* /content/data_split256/train/

In [ ]:
# !cp /content/data_split256/valid/* /content/data_split256/train/

### 데이터 개수 확인

In [ ]:
# 기존 valid, test 데이터를 전부 train에 활용

print(len(os.listdir('/content/data_split256/train/')))
print(len(os.listdir('/content/data_split256/valid/')))
print(len(os.listdir('/content/data_split256/test/')))

121260
22444
10976


### Preprocessing 함수

In [ ]:
# def get_thresh(img, pm):
#     w, h, _ = img.shape
#     cx, cy = int(w/2), int(h/2)
#     center = img[cx - 30:cx+30, cy-30: cy+30, :]
#     # center = np.mean(center, axis=0)
#     # center = np.mean(center, axis=0)
#     lower = np.array(np.clip(center-pm,0, 255 ))
#     upper = np.array(np.clip(center+pm,0, 255 ))

#     # upper = np.array(np.clip(center.max(axis=0).max(axis=0) + [40, 50, 50], 0, 255), dtype=np.uint8)
#     upper = np.array(np.clip(center.max(axis=0).max(axis=0) * (2-pm), 0, 255), dtype=np.uint8)
#     # upper = np.array([255, 255, 255], dtype=np.uint8)
#     lower = np.clip(np.array(center.min(axis=0).min(axis=0) * pm, dtype=np.uint8), 0, 255)
#     return lower, upper

In [ ]:
# def get_mask(path):
#     # imshow("original", img)
#     img = cv2.imread(path, cv2.IMREAD_COLOR)

#     hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)
#     rgb = img

#     rgb_lower, rgb_upper = get_thresh(rgb, np.array([1, 1*.90, 1]))
#     rgb_mask = cv2.inRange(rgb, rgb_lower, rgb_upper)

#     ycrcb_lower, ycrcb_upper = get_thresh(ycrcb, np.array([1, 1, 1])*.95)
#     ycrcb_mask = cv2.inRange(ycrcb, ycrcb_lower, ycrcb_upper)

#     hsv_lower, hsv_upper = get_thresh(hsv, .95)
#     hsv_mask = cv2.inRange(hsv, hsv_lower, hsv_upper)

#     hsv_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,20))
#     hsv_finger = cv2.morphologyEx(hsv_mask, cv2.MORPH_OPEN, hsv_kernel)

#     rgb_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,20))
#     rgb_finger = cv2.morphologyEx(rgb_mask, cv2.MORPH_OPEN, rgb_kernel)
    
#     ycrcb_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20,20))
#     ycrcb_finger = cv2.morphologyEx(ycrcb_mask, cv2.MORPH_OPEN, ycrcb_kernel)
    
#     mask = cv2.bitwise_and(rgb_finger, hsv_finger)
#     mask = cv2.bitwise_and(mask, ycrcb_finger)

#     # finger=cv2.bitwise_and(img, img, mask = rgb_finger)
#     # finger=cv2.bitwise_and(finger, finger, mask = hsv_finger)
#     # finger=cv2.bitwise_and(finger, finger, mask = ycrcb_finger)

#     finger = cv2.bitwise_and(img, img, mask = mask)

#     return finger, mask

In [ ]:
# _sigma_conv = (3.0/2.0)/((6*math.log(10))**0.5)
# # sigma is adjusted according to the ridge period, so that the filter does not contain more than three effective peaks 
# def _gabor_sigma(ridge_period):
#     return _sigma_conv * ridge_period

# def _gabor_size(ridge_period):
#     p = int(round(ridge_period * 2 + 1))
#     if p % 2 == 0:
#         p += 1
#     return (p, p)

# def gabor_kernel(period, orientation):
#     f = cv2.getGaborKernel(_gabor_size(period), _gabor_sigma(period), np.pi/2 - orientation, period, gamma = 1, psi = 0)
#     f /= f.sum()
#     f -= f.mean()
#     return f

In [ ]:
# def Enhancement(fingerprint):
#     # Calculate the local gradient (using Sobel filters)
#     gx, gy = cv2.Sobel(fingerprint, cv2.CV_32F, 1, 0), cv2.Sobel(fingerprint, cv2.CV_32F, 0, 1)
    
#     # Calculate the magnitude of the gradient for each pixel
#     gx2, gy2 = gx**2, gy**2

    
#     W = (23, 23)
#     gxx = cv2.boxFilter(gx2, -1, W, normalize = False)
#     gyy = cv2.boxFilter(gy2, -1, W, normalize = False)
#     gxy = cv2.boxFilter(gx * gy, -1, W, normalize = False)
#     gxx_gyy = gxx - gyy
#     gxy2 = 2 * gxy

#     orientations = (cv2.phase(gxx_gyy, -gxy2) + np.pi) / 2 # '-' to adjust for y axis direction
    

#     region = fingerprint[80:180,80:180]

#     # before computing the x-signature, the region is smoothed to reduce noise
#     smoothed = cv2.blur(region, (5,5), -1)
#     xs = np.sum(smoothed, 1) # the x-signature of the region

#     # Find the indices of the x-signature local maxima
#     local_maxima = np.nonzero(np.r_[False, xs[1:] > xs[:-1]] & np.r_[xs[:-1] >= xs[1:], False])[0]

#     # Calculate all the distances between consecutive peaks
#     distances = local_maxima[1:] - local_maxima[:-1]

#     # Estimate the ridge line period as the average of the above distances
#     ridge_period = np.average(distances)

#     # Create the filter bank
#     or_count = 8
#     gabor_bank = [gabor_kernel(ridge_period, o) for o in np.arange(0, np.pi, np.pi/or_count)]

#     # Filter the whole image with each filter
#     # Note that the negative image is actually used, to have white ridges on a black background as a result
#     nf = 255-fingerprint
#     all_filtered = np.array([cv2.filter2D(nf, cv2.CV_32F, f) for f in gabor_bank])

#     y_coords, x_coords = np.indices(fingerprint.shape)
#     # For each pixel, find the index of the closest orientation in the gabor bank
#     orientation_idx = np.round(((orientations % np.pi) / np.pi) * or_count).astype(np.int32) % or_count
#     # Take the corresponding convolution result for each pixel, to assemble the final result
#     filtered = all_filtered[orientation_idx, y_coords, x_coords]
#     # Convert to gray scale and apply the mask
#     enhanced = mask & np.clip(filtered, 0, 255).astype(np.uint8)

#     return enhanced

In [ ]:
# def Segmentation(path) :
#     # 이미지 파일 읽기
#     img = cv2.imread(path, cv2.IMREAD_COLOR)

#     # 이미지 색 바꾸기
#     # hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

#     # lower = np.array([0,48,80], dtype="uint8")
#     # upper = np.array([20,255,255], dtype="uint8")

#     ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCR_CB)

#     upper = np.array([255, 173, 127])
#     lower = np.array([0, 133, 77])

#     mask = cv2.inRange(ycrcb, lower, upper)

#     kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))
#     finger_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

#     finger = cv2.bitwise_and(img, img, mask = finger_mask)
  
#     return finger, finger_mask


In [ ]:
# def create_mask(width, height, mask_width, mask_height, x=None, y=None):
#     mask = np.zeros((height, width))
#     # mask_x = x if x is not None else random.randint(0, width - mask_width)
#     # mask_y = y if y is not None else random.randint(0, height - mask_height)
#     # mask[mask_y:mask_y + mask_height, mask_x:mask_x + mask_width] = 1
#     cv2.rectangle(mask, (mask_width // 2, mask_height // 2), ((3 * mask_width // 2), (3 * mask_height // 2)), 255, -1)
#     # none_mask = mask[mask_height // 2:(3 * mask_height // 2), mask_width // 2:(3 * mask_width // 2)] = 1
#     return mask

In [ ]:
# def extract_edge(img):
#     copy = img.copy()
#     effect = cv2.convertScaleAbs(copy, alpha=0.8, beta=25)
#     gray = cv2.cvtColor(effect, cv2.COLOR_RGB2GRAY)
#     equ = cv2.equalizeHist(gray)
#     norm = cv2.normalize(equ, None, 0, 256, cv2.NORM_MINMAX)
#     result = cv2.adaptiveThreshold(norm, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 15, 2)

#     # blur = cv2.blur(thres,(5,5))
#     # effect2 = cv2.convertScaleAbs(blur, alpha=1.7, beta=-40)
#     # blur2 = cv2.GaussianBlur(effect2,(15,15),0)
#     # addweight = cv2.addWeighted(blur2, 0.5, effect2, -0.5, 0.0)
#     # ret, thresh = cv2.threshold(addweight ,10, 255, cv2.THRESH_BINARY)

#     return result

### Mask 및 Edge 파일 생성

#### train, valid data

In [ ]:
base_dir = "/content/data_split256"
sub_dirs = ["train", "valid"]

dst = "/content/data_split_edge"
if not os.path.isdir(dst):
    os.mkdir(dst)

for sub in tqdm(sub_dirs): 
    current = os.path.join(base_dir, sub)
    list_dir = os.listdir(current)
    list_dir = [file for file in list_dir if file.endswith((".jpg", ".png"))]


    mask_dir = base_dir + "/mask/" + sub
    if not os.path.isdir(mask_dir):
        os.mkdir(mask_dir)

    dst = os.path.join("/content/data_split_edge", sub)
    if not os.path.isdir(dst):
        os.mkdir(dst)

    for i in tqdm(list_dir):
        joined = os.path.join(current, i)

        # 이미지에서 mask 생성
        img, mask = get_mask(joined)
        # Mask 파일 저장
        new_mask_dir = os.path.join(mask_dir, i)
        cv2.imwrite(new_mask_dir, mask)

        # Edge 추출
        edge = extract_edge(img)
        
        # Fingerprint Enhancement
        result = Enhancement(edge)
        
        # Edge 파일 저장
        new_dir = os.path.join(dst, i)
        cv2.imwrite(new_dir, result)

### test data

In [ ]:
base_dir = "/content/images"
sub_dirs = ["test"]

dst = "/content/data_split_edge"
if not os.path.isdir(dst):
    os.mkdir(dst)

mask_dir = "/content/images/mask"
if not os.path.isdir(mask_dir):
    os.mkdir(mask_dir)


for sub in tqdm(sub_dirs): 
    current = base_dir
    list_dir = os.listdir(current)
    list_dir = [file for file in list_dir if file.endswith(".jpg")]

    dst = os.path.join("/content/data_split_edge", sub)
    if not os.path.isdir(dst):
        os.mkdir(dst)
    
    for i in tqdm(list_dir):
        joined = os.path.join(current, i)

        # 이미지에서 mask 생성
        img, mask = get_mask(joined)
        # Mask 파일 저장
        new_mask_dir = os.path.join(mask_dir, i)
        cv2.imwrite(new_mask_dir, mask)

        # Edge 추출
        edge = extract_edge(img)

        # Fingerprint Enhancement
        result = Enhancement(thres)
        
        # Edge 파일 저장
        new_dir = os.path.join(dst, i)
        cv2.imwrite(new_dir, result)

### 생성 파일 압축 및 드라이브 저장

In [ ]:
# test image(mask 포함)
! zip -r test.zip images

In [ ]:
! cp test.zip /content/drive/MyDrive/edge-connect

In [ ]:
# edge
! zip -r data_split_edge.zip data_split_edge

In [ ]:
! cp data_split_edge.zip /content/drive/MyDrive/edge-connect

In [ ]:
# train, valid image(mask 포함)
! zip -r data_split256.zip data_split256

In [ ]:
! cp data_split256.zip /content/drive/MyDrive/edge-connect